In [1]:
import pandas as pd
import json
import glob

In [2]:
pd.set_option('display.max_columns', None)

### Format BODACC

In [6]:
df_bodacc = pd.read_csv(r"C:\Users\RémiJAYLET\Documents\City&You\0_data\bodacc\annonces-commerciales.csv",
                        usecols=['id', 'dateparution', 'numeroannonce',
                                 'region_code', 'numerodepartement', 'ville',
                                 'tribunal', 'commercant',
                                 'registre', 'listepersonnes',
                                 'jugement'],
                        dtype={'id': str,
                               'numeroannonce': str,
                               'region_code': str,
                               'numerodepartement': str,
                               'registre': str},
                        sep=';',
                        encoding='utf-8-sig')

In [8]:
# get jugement info
df_bodacc = df_bodacc.dropna(subset='jugement')
df_bodacc['jugement_nature'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['nature'] if 'nature' in json.loads(x) else '')
df_bodacc['jugement_famille'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['famille'] if 'famille' in json.loads(x) else '')
df_bodacc['jugement_date'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['date'] if 'date' in json.loads(x) else '')
df_bodacc['jugement_complement'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['complementJugement'] if 'complementJugement' in json.loads(x) else '')
df_bodacc = df_bodacc.drop('jugement', axis=1)

C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_22640\2905835236.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bodacc['jugement_nature'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['nature'] if 'nature' in json.loads(x) else '')
C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_22640\2905835236.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bodacc['jugement_famille'] = df_bodacc['jugement'].apply(lambda x: json.loads(x)['famille'] if 'famille' in json.loads(x) else '')
C:\Users\Ré

In [9]:
df_bodacc['jugement_famille'].unique()

array(["Jugement d'ouverture", 'Avis de dépôt', 'Extrait de jugement',
       'Jugement de clôture', 'Jugement prononçant',
       'Rétractation sur tierce opposition', 'Loi de 1967',
       "Arrêt de la Cour d'Appel", 'Jugement prononÃ§ant',
       'Avis de dÃ©pÃ´t', 'Jugement de clÃ´ture',
       "ArrÃªt de la Cour d'Appel"], dtype=object)

In [8]:
# fix error import
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace('Jugement prononÃ§ant', 'Jugement prononçant')
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace('Avis de dÃ©pÃ´t', 'Avis de dépôt')
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace('Jugement de clÃ´ture', 'Jugement de clôture')
df_bodacc['jugement_famille'] = df_bodacc['jugement_famille'].replace("ArrÃªt de la Cour d'Appel", "Arrêt de la Cour d'Appel")

In [9]:
# get rid of null siren with no info
df_bodacc = df_bodacc.dropna(subset='registre')
df_bodacc['registre'] = df_bodacc['registre'].astype(str)
df_bodacc['registre'] = df_bodacc['registre'].apply(lambda x : x.replace('000 000 000,000000000,', ''))
df_bodacc = df_bodacc[df_bodacc['registre'] != '000 000 000,000000000']

In [10]:
# extract SIREN
df_bodacc['SIREN'] = df_bodacc['registre'].apply(lambda x: x.split(',')[0].replace(' ', ''))
df_bodacc = df_bodacc.drop('registre', axis=1)

In [11]:
# format dates
df_bodacc['dateparution'] = pd.to_datetime(df_bodacc['dateparution'])
df_bodacc['jugement_date'] = pd.to_datetime(df_bodacc['jugement_date'], format='mixed', dayfirst=True, errors='ignore')

In [12]:
df_bodacc = df_bodacc.sort_values(by=['dateparution', 'SIREN'])

In [13]:
len(df_bodacc)

2451404

In [14]:
len(df_bodacc.SIREN.unique())

900242

In [15]:
# keep only relevant categories
df_bodacc = df_bodacc.loc[(df_bodacc['jugement_famille'] != 'Rétractation sur tierce opposition') & 
                          (df_bodacc['jugement_famille'] != 'Loi de 1967') &
                          (df_bodacc['jugement_famille'] != "Arrêt de la Cour d'Appel")]

In [16]:
# convert to simple sub categories of judgements
df_bodacc.loc[(df_bodacc['jugement_famille'] == 'Jugement prononçant') & (df_bodacc['jugement_nature'] != 'Jugement de conversion en liquidation judiciaire'), 'jugement_nature'] = 'Autre jugement'
df_bodacc.loc[(df_bodacc['jugement_famille'] == 'Jugement de clôture') & (df_bodacc['jugement_nature'] != "Jugement de clôture pour insuffisance d'actif"), 'jugement_nature'] = 'Autre jugement'
df_bodacc.loc[(df_bodacc['jugement_famille'] == "Jugement d'ouverture") & ((df_bodacc['jugement_nature'] != "Jugement d'ouverture de liquidation judiciaire") & (df_bodacc['jugement_nature'] != "Jugement d'ouverture d'une procédure de redressement judiciaire")), 'jugement_nature'] = 'Autre jugement'
df_bodacc.loc[(df_bodacc['jugement_famille'] == "Avis de dépôt") & (df_bodacc['jugement_nature'] != "Dépôt de l'état des créances"), 'jugement_nature'] = 'Autre jugement'

In [17]:
# simplify jugemetn nature for ouverture
df_bodacc.loc[df_bodacc['jugement_nature'] == "Jugement d'ouverture de liquidation judiciaire", "jugement_famille"] = "Jugement d'ouverture de liquidation"
df_bodacc.loc[df_bodacc['jugement_nature'] == "Jugement d'ouverture d'une procédure de redressement judiciaire", "jugement_famille"] = "Jugement d'ouverture de redressement"
df_bodacc.loc[df_bodacc['jugement_nature'] == "Autre jugement", "jugement_famille"] = "Jugement d'ouverture autre"

In [18]:
len(df_bodacc)

2443796

In [19]:
df_bodacc = df_bodacc[df_bodacc.numerodepartement.isin(['75', '77', '78', '91', '92', '93', '94', '95'])]

In [20]:
len(df_bodacc)

481844

### Format BODACC liquidation

In [21]:
df_bodacc_liq = df_bodacc[df_bodacc.jugement_famille == "Jugement d'ouverture de liquidation"].drop_duplicates('SIREN', keep='last')
df_bodacc_red = df_bodacc[df_bodacc.jugement_famille == "Jugement d'ouverture de redressement"].drop_duplicates('SIREN', keep='last')

In [22]:
siren_date = df_bodacc[(df_bodacc.dateparution > '2020-01-01')]
# format other type of date TO FIX
siren_date['year'] = siren_date['dateparution'].apply(lambda x: str(x)[:4])
siren_date = siren_date[siren_date['year'].isin(['2020', '2021', '2022', '2023'])]

C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_10116\1735739652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siren_date['year'] = siren_date['dateparution'].apply(lambda x: str(x)[:4])


In [23]:
df_bodacc_liq = df_bodacc_liq[df_bodacc_liq.SIREN.isin(list(siren_date.SIREN))]
df_bodacc_red = df_bodacc_red[df_bodacc_red.SIREN.isin(list(siren_date.SIREN))]

In [24]:
df_bodacc_liq

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,listepersonnes,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN
2560262,A200800932929,2008-01-06,2929,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,ETS JEAN GUERIN,Paris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,5 mai 2008,"Jugement prononçant la liquidation judiciaire,...",592042519
2448545,A200802001919,2008-02-11,1919,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,ARTS-IMMO,Rueil-Malmaison,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,16 octobre 2008,"Jugement prononçant la liquidation judiciaire,...",397994914
2167934,A200800373636,2008-02-29,3636,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,SYLANDCO,Paris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,10 janvier 2008,"Jugement prononçant la liquidation judiciaire,...",481753861
2562729,A200801361683,2008-03-08,1683,93,11.0,TRIBUNAL DE GRANDE INSTANCE DE BOBIGNY,"PROSZENUCK, Sylvie",Saint-Denis,"{""personne"": {""typePersonne"": ""pp"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,3 juillet 2008,Jugement d'ouverture de la liquidation judicia...,384262960
2158501,A200800466080,2008-03-18,6080,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,"BARBAR, Ibrahim",Pantin,"{""personne"": {""typePersonne"": ""pp"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 février 2008,"Jugement prononçant la liquidation judiciaire,...",450121975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730633,A202301002293,2023-05-25,2293,92,11.0,Greffe du Tribunal de Commerce de Nanterre,NAS 92,Asnières-sur-Seine,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",893263277
2510034,A202301002292,2023-05-25,2292,92,11.0,Greffe du Tribunal de Commerce de Nanterre,MURS EN COULEUR,Nanterre,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",893486308
2669859,A202301002213,2023-05-25,2213,77,11.0,Greffe du Tribunal de Commerce de Melun,ISOBAT,Vaux-le-Pénil,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-15,"Jugement prononçant la liquidation judiciaire,...",899286405
2509860,A202301002158,2023-05-25,2158,77,11.0,Greffe du Tribunal de Commerce de Meaux,BFCG Immo,Serris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-09,Jugement prononçant la liquidation judiciaire ...,899693543


In [25]:
df_bodacc_red

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,listepersonnes,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN
1076938,A200800932918,2008-01-06,2918,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,LA LOCOMOTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,22 avril 2008,Jugement prononçant l'ouverture d'une procédur...,338706047
2673459,A200800932916,2008-01-06,2916,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,PRONUPTIA DE PARIS,Paris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,29 avril 2008,Jugement prononçant l'ouverture d'une procédur...,582065025
2269785,A200801351682,2008-01-08,1682,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,CABINET PORTELLI,Boulogne-Billancourt,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,17 juillet 2008,Jugement prononçant l'ouverture d'une procédur...,395309966
2597504,A200801772063,2008-01-10,2063,95,11.0,TRIBUNAL DE COMMERCE DE PONTOISE,"EUROPEENNE DE CARROSSERIE LAURENT, E. C. L.",Goussainville,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,15 septembre 2008,Jugement prononçant l'ouverture d'une procédur...,350188306
2694901,A200800385101,2008-02-03,5101,95,11.0,TRIBUNAL DE COMMERCE DE PONTOISE,"RINAGEL, Marie Claude",Sarcelles,"{""personne"": {""typePersonne"": ""pp"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,4 février 2008,Jugement prononçant l'ouverture d'une procédur...,422048512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730624,A202301002155,2023-05-25,2155,77,11.0,Greffe du Tribunal de Commerce de Meaux,AUTOMATIC CARS,Le Blanc Mesnil,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-09,Jugement prononçant l'ouverture d'une procédur...,824818496
2710795,A202301002163,2023-05-25,2163,77,11.0,Greffe du Tribunal de Commerce de Meaux,ECURIE MARNE ET GONDOIRE,Conches-sur-Gondoire,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-09,Jugement prononçant l'ouverture d'une procédur...,825323041
2693410,A202301002160,2023-05-25,2160,77,11.0,Greffe du Tribunal de Commerce de Meaux,CAM IT CONSULTING,Chauconin Neufmontiers,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-09,Jugement prononçant l'ouverture d'une procédur...,830663522
2633638,A202301002245,2023-05-25,2245,78,11.0,Greffe du Tribunal de Commerce de Versailles,SAIDI TRANSPORT,Chatou,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-16,Jugement prononçant l'ouverture d'une procédur...,844353722


In [26]:
df_bodacc_red = df_bodacc_red[~df_bodacc_red.SIREN.isin(list(df_bodacc_liq.SIREN.unique()))]

In [27]:
df_bodacc_red

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,listepersonnes,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN
1076938,A200800932918,2008-01-06,2918,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,LA LOCOMOTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,22 avril 2008,Jugement prononçant l'ouverture d'une procédur...,338706047
2673459,A200800932916,2008-01-06,2916,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,PRONUPTIA DE PARIS,Paris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,29 avril 2008,Jugement prononçant l'ouverture d'une procédur...,582065025
2597504,A200801772063,2008-01-10,2063,95,11.0,TRIBUNAL DE COMMERCE DE PONTOISE,"EUROPEENNE DE CARROSSERIE LAURENT, E. C. L.",Goussainville,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,15 septembre 2008,Jugement prononçant l'ouverture d'une procédur...,350188306
2694901,A200800385101,2008-02-03,5101,95,11.0,TRIBUNAL DE COMMERCE DE PONTOISE,"RINAGEL, Marie Claude",Sarcelles,"{""personne"": {""typePersonne"": ""pp"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,4 février 2008,Jugement prononçant l'ouverture d'une procédur...,422048512
2357152,A200800384901,2008-02-03,4901,91,11.0,TRIBUNAL DE COMMERCE D'EVRY,G. A. D. L.,Mennecy,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,4 février 2008,Jugement prononçant l'ouverture d'une procédur...,487625329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730624,A202301002155,2023-05-25,2155,77,11.0,Greffe du Tribunal de Commerce de Meaux,AUTOMATIC CARS,Le Blanc Mesnil,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-09,Jugement prononçant l'ouverture d'une procédur...,824818496
2710795,A202301002163,2023-05-25,2163,77,11.0,Greffe du Tribunal de Commerce de Meaux,ECURIE MARNE ET GONDOIRE,Conches-sur-Gondoire,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-09,Jugement prononçant l'ouverture d'une procédur...,825323041
2693410,A202301002160,2023-05-25,2160,77,11.0,Greffe du Tribunal de Commerce de Meaux,CAM IT CONSULTING,Chauconin Neufmontiers,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-09,Jugement prononçant l'ouverture d'une procédur...,830663522
2633638,A202301002245,2023-05-25,2245,78,11.0,Greffe du Tribunal de Commerce de Versailles,SAIDI TRANSPORT,Chatou,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture d'une procédure de redres...,Jugement d'ouverture de redressement,2023-05-16,Jugement prononçant l'ouverture d'une procédur...,844353722


In [28]:
cloture_list = list(df_bodacc[df_bodacc.jugement_famille == "Jugement de clôture"].drop_duplicates('SIREN', keep='last').SIREN)

In [29]:
df_bodacc_liq = df_bodacc_liq[~df_bodacc_liq['SIREN'].isin(cloture_list)]
df_bodacc_red = df_bodacc_red[~df_bodacc_red['SIREN'].isin(cloture_list)]

In [30]:
len(df_bodacc_liq)

13089

In [31]:
len(df_bodacc_red)

6253

In [32]:
df_bodacc_liq.head(3)

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,listepersonnes,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN
2448545,A200802001919,2008-02-11,1919,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,ARTS-IMMO,Rueil-Malmaison,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,16 octobre 2008,"Jugement prononçant la liquidation judiciaire,...",397994914
2553698,A200800923221,2008-05-31,3221,92,11.0,TRIBUNAL DE COMMERCE DE NANTERRE,SHS FRANCE,Boulogne-Billancourt,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,13 mai 2008,"Jugement prononçant la liquidation judiciaire,...",493286553
2415614,A200801921873,2008-10-22,1873,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,DATASCOR,Paris,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,29 septembre 2008,"Jugement prononçant la liquidation judiciaire,...",323505743


### Import other bases

In [33]:
# get SIREN of companies that are PM
SIREN = pd.read_csv('../0_data/base_sirene/StockUniteLegale_utf8.csv', 
                    usecols=['siren', 'denominationUniteLegale', 'prenom1UniteLegale'],
                    dtype = {'siren': str})
SIREN = SIREN[SIREN.prenom1UniteLegale.isna()]

C:\Users\RémiJAYLET\AppData\Local\Temp\ipykernel_10116\706957035.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  SIREN = pd.read_csv('../0_data/base_sirene/StockUniteLegale_utf8.csv',


In [34]:
df_bodacc_liq = df_bodacc_liq[df_bodacc_liq.SIREN.isin(list(SIREN.siren))]

In [35]:
df_bodacc_red = df_bodacc_red[df_bodacc_red.SIREN.isin(list(SIREN.siren))]

In [36]:
len(df_bodacc_liq)

12670

In [37]:
len(df_bodacc_red)

5807

In [38]:
df_parc1 = pd.concat([pd.read_csv(i,
                                  sep=';',
                                  encoding='latin',
                                  low_memory=False,
                                  dtype={'Département (Champ géographique)' : str,
                                         'Code Direction (Champ géographique)' : str,
                                         'Code Commune (Champ géographique)' : str,
                                         'Préfixe (Références cadastrales)' : str,
                                         'Section (Références cadastrales)' : str,
                                         'N° plan (Références cadastrales)' : str}) for i in glob.glob(
    "../0_data/parcelles/Fichier des parcelles (situation 2022)-dept 01 à 61/Fichier des parcelles (situation 2022)-dpts 01 à 61/*.txt", recursive=True)])

df_parc2 = pd.concat([pd.read_csv(i, sep=';',
                                  encoding='latin',
                                  low_memory=False,
                                  dtype={'Département (Champ géographique)' : str,
                                         'Code Direction (Champ géographique)' : str,
                                         'Code Commune (Champ géographique)' : str,
                                         'Préfixe (Références cadastrales)' : str,
                                         'Section (Références cadastrales)' : str,
                                         'N° plan (Références cadastrales)' : str}) for i in glob.glob(
    "../0_data/parcelles/Fichier des parcelles (situation 2022)-dept 62 à 976/Fichier des parcelles (situation 2022)-dpts 62 à 976/*.txt", recursive=True)])

df_parc = pd.concat([df_parc1, df_parc2], axis=0)

In [39]:
df_loc = pd.concat([pd.read_csv(i,
                                sep=';',
                                encoding='latin',
                                low_memory=False,
                                dtype={'Département (Champ géographique)' : str,
                                         'Code Direction (Champ géographique)' : str,
                                         'Code Commune (Champ géographique)' : str,
                                         'Préfixe (Références cadastrales)' : str,
                                         'Section (Références cadastrales)' : str,
                                         'N° plan (Références cadastrales)' : str}) for i in glob.glob(
    "../0_data/locaux/Fichier des locaux/*.txt",
    recursive=True)])

In [41]:
SIRET = pd.read_csv('../0_data/base_sirene/StockEtablissement_utf8.csv',
                    usecols=['siren', 'siret', 'libelleVoieEtablissement', 'libelleCommuneEtablissement'],
                    dtype = {'siren': str, 'siret': str, 'libelleVoieEtablissement': str, 'libelleCommuneEtablissement': str})

MemoryError: Unable to allocate 279. MiB for an array with shape (36522041,) and data type object

### Analysis

In [42]:
df = df_bodacc_liq.copy()
df2 = df_bodacc_red.copy()

In [44]:
del df_bodacc

#### With parcelles

In [45]:
parc_bodacc = df.merge(df_parc, left_on='SIREN',
                       right_on = 'N° SIREN (Propriétaire(s) parcelle)',
                       how='left')

In [46]:
parc_bodacc = parc_bodacc[~parc_bodacc['N° SIREN (Propriétaire(s) parcelle)'].isna()]
parc_bodacc = parc_bodacc.drop_duplicates(['SIREN', 'jugement_date', 'Département (Champ géographique)', 
                                           'Code Commune (Champ géographique)', 'Section (Références cadastrales)',
                                           'Préfixe (Références cadastrales)', 'N° plan (Références cadastrales)'])

In [47]:
parc_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,listepersonnes,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),N° de voirie (Adresse parcelle),Indice de répétition (Adresse parcelle),Code voie MAJIC (Adresse parcelle),Code voie rivoli (Adresse parcelle),Nature voie (Adresse parcelle),Nom voie (Adresse parcelle),Contenance (Caractéristiques parcelle),SUF (Evaluation SUF),Nature culture (Evaluation SUF),Contenance (Evaluation SUF),Code droit (Propriétaire(s) parcelle),N° MAJIC (Propriétaire(s) parcelle),N° SIREN (Propriétaire(s) parcelle),Groupe personne (Propriétaire(s) parcelle),Forme juridique (Propriétaire(s) parcelle),Forme juridique abrégée (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle)
19,A201200192285,2012-01-27,2285,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,FONCIERE RICHELIEU,Montreuil,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,12 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",333697795,93,0,027,LA COURNEUVE,,O,0343,NaN,NaN,180.0,7125,RUE,PASCAL,690.0,NaN,S,690.0,P,PBBLQ7,333697795,0,5710,SAS,FONCIERE RICHELIEU
20,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0253,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,8639.0,NaN,AG,8639.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
21,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0254,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,812.0,NaN,AB,812.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
22,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0255,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,1042.0,NaN,AG,1042.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
23,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0256,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,6467.0,NaN,AB,6467.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13424,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,,AB,0721,NaN,NaN,183.0,B185,NaN,LE VILLAGE,1812.0,NaN,AG,1812.0,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE
13425,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,"{""personne"": {""typePerso

In [48]:
print('Number of parcelles concerned :', len(parc_bodacc))

Number of parcelles concerned : 905


In [49]:
parc_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,listepersonnes,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),N° de voirie (Adresse parcelle),Indice de répétition (Adresse parcelle),Code voie MAJIC (Adresse parcelle),Code voie rivoli (Adresse parcelle),Nature voie (Adresse parcelle),Nom voie (Adresse parcelle),Contenance (Caractéristiques parcelle),SUF (Evaluation SUF),Nature culture (Evaluation SUF),Contenance (Evaluation SUF),Code droit (Propriétaire(s) parcelle),N° MAJIC (Propriétaire(s) parcelle),N° SIREN (Propriétaire(s) parcelle),Groupe personne (Propriétaire(s) parcelle),Forme juridique (Propriétaire(s) parcelle),Forme juridique abrégée (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle)
19,A201200192285,2012-01-27,2285,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,FONCIERE RICHELIEU,Montreuil,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,12 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",333697795,93,0,027,LA COURNEUVE,,O,0343,NaN,NaN,180.0,7125,RUE,PASCAL,690.0,NaN,S,690.0,P,PBBLQ7,333697795,0,5710,SAS,FONCIERE RICHELIEU
20,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0253,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,8639.0,NaN,AG,8639.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
21,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0254,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,812.0,NaN,AB,812.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
22,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0255,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,1042.0,NaN,AG,1042.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
23,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,,AA,0256,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,6467.0,NaN,AB,6467.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13424,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,,AB,0721,NaN,NaN,183.0,B185,NaN,LE VILLAGE,1812.0,NaN,AG,1812.0,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE
13425,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,"{""personne"": {""typePerso

In [53]:
del df_bodacc_liq, df_bodacc_red

In [55]:
parc_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,listepersonnes,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),N° de voirie (Adresse parcelle),Indice de répétition (Adresse parcelle),Code voie MAJIC (Adresse parcelle),Code voie rivoli (Adresse parcelle),Nature voie (Adresse parcelle),Nom voie (Adresse parcelle),Contenance (Caractéristiques parcelle),SUF (Evaluation SUF),Nature culture (Evaluation SUF),Contenance (Evaluation SUF),Code droit (Propriétaire(s) parcelle),N° MAJIC (Propriétaire(s) parcelle),N° SIREN (Propriétaire(s) parcelle),Groupe personne (Propriétaire(s) parcelle),Forme juridique (Propriétaire(s) parcelle),Forme juridique abrégée (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle)
19,A201200192285,2012-01-27,2285,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,FONCIERE RICHELIEU,Montreuil,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,12 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",333697795,93,0,027,LA COURNEUVE,000,0O,0343,NaN,NaN,180.0,7125,RUE,PASCAL,690.0,NaN,S,690.0,P,PBBLQ7,333697795,0,5710,SAS,FONCIERE RICHELIEU
20,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0253,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,8639.0,NaN,AG,8639.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
21,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0254,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,812.0,NaN,AB,812.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
22,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0255,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,1042.0,NaN,AG,1042.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
23,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,"{""personne"": {""typePersonne"": ""pm"", ""activite""...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,1,449,ORCHIES,000,AA,0256,NaN,NaN,134.0,0031,RUE,ATTILIO BERNASCONI,6467.0,NaN,AB,6467.0,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13424,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,"{""personne"": {""typePersonne"": ""pm"", ""numeroImm...",Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,000,AB,0721,NaN,NaN,183.0,B185,NaN,LE VILLAGE,1812.0,NaN,AG,1812.0,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE
13425,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,"{""perso

In [56]:
parc_bodacc['Préfixe (Références cadastrales)'] = parc_bodacc['Préfixe (Références cadastrales)'].replace('   ', '000')
parc_bodacc['Section (Références cadastrales)'] = parc_bodacc['Section (Références cadastrales)'].apply(lambda x: '0' + x if len(str(x)) == 1 else x)
parc_bodacc['parcelle_id'] = parc_bodacc['Département (Champ géographique)'] + parc_bodacc['Code Commune (Champ géographique)'] + parc_bodacc['Préfixe (Références cadastrales)']  + parc_bodacc['Section (Références cadastrales)'] + parc_bodacc['N° plan (Références cadastrales)']
first_column = parc_bodacc.pop('parcelle_id')
parc_bodacc.insert(0, 'parcelle_id', first_column)


In [60]:
parc_bodacc.to_csv('20230530_propriete_parcelles_bodacc_liq_idf.csv', sep=';', encoding='utf-8-sig', index=False)

In [64]:
parc_bodacc.to_csv('20230530_propriete_parcelles_bodacc_red_idf.csv', sep=';', encoding='utf-8-sig')

In [93]:
print('Number of companies concerned :', len(parc_bodacc.SIREN.unique()))

Number of companies concerned : 139


In [94]:
parc_bodacc['Préfixe (Références cadastrales)'] = parc_bodacc['Section (Références cadastrales)'].apply(lambda x: '000' if len(x) == 2 else '0000')
parc_bodacc['parcelle_id'] = parc_bodacc['Département (Champ géographique)'] + parc_bodacc['Code Commune (Champ géographique)'] + parc_bodacc['Préfixe (Références cadastrales)'] + parc_bodacc['Section (Références cadastrales)']  + parc_bodacc['N° plan (Références cadastrales)']

In [95]:
list_cities = ['ALFORTVILLE', 'CHAMPIGNY SUR MARNE', 'FONTENAY SOUS BOIS',
               'ST MAUR DES FOSSES', 'ASNIERES-SUR-SEINE', 'CLICHY', 'LIVRY GARGAN']

parc_bodacc[parc_bodacc['Nom Commune (Champ géographique)'].isin(list_cities)]

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),N° de voirie (Adresse parcelle),Indice de répétition (Adresse parcelle),Code voie MAJIC (Adresse parcelle),Code voie rivoli (Adresse parcelle),Nature voie (Adresse parcelle),Nom voie (Adresse parcelle),Contenance (Caractéristiques parcelle),SUF (Evaluation SUF),Nature culture (Evaluation SUF),Contenance (Evaluation SUF),Code droit (Propriétaire(s) parcelle),N° MAJIC (Propriétaire(s) parcelle),N° SIREN (Propriétaire(s) parcelle),Groupe personne (Propriétaire(s) parcelle),Forme juridique (Propriétaire(s) parcelle),Forme juridique abrégée (Propriétaire(s) parcelle),Dénomination (Propriétaire(s) parcelle),parcelle_id
273,A201602462390,2016-12-18,2390,94,11.0,TRIBUNAL DE COMMERCE DE CRÉTEIL,CABINET LAMBERT,Fontenay-sous-Bois,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2016-12-07,"Jugement prononçant la liquidation judiciaire,...",342376928,94,0,033,FONTENAY SOUS BOIS,000,BQ,0002,77.0,NaN,342.0,7140,RUE,PASTEUR,836.0,NaN,S,836.0,G,PBBBF9,342376928,0,5710,SAS,CABINET LAMBERT,94033000BQ0002
274,A201602462390,2016-12-18,2390,94,11.0,TRIBUNAL DE COMMERCE DE CRÉTEIL,CABINET LAMBERT,Fontenay-sous-Bois,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2016-12-07,"Jugement prononçant la liquidation judiciaire,...",342376928,94,0,033,FONTENAY SOUS BOIS,0000,R,0015,20.0,NaN,185.0,3160,AV,ERNEST RENAN,211.0,NaN,S,211.0,S,PBBBF9,342376928,0,5710,SAS,CABINET LAMBERT,940330000R0015
2769,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0068,56.0,NaN,110.0,7130,RUE,DE PARIS,288.0,NaN,S,288.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0068
2770,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0069,71.0,NaN,70.0,4420,RUE,HENRI BARBUSSE,354.0,NaN,S,354.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0069
2771,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0148,75.0,NaN,70.0,4420,RUE,HENRI BARBUSSE,280.0,NaN,S,280.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0148
2772,A202100024350,2021-01-05,4350,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,CABINET ROLET-BONTEMPS,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2020-12-17,"Jugement prononçant la liquidation judiciaire,...",331153676,92,1,024,CLICHY,0000,Q,0179,75.0,NaN,70.0,4420,RUE,HENRI BARBUSSE,290.0,NaN,S,290.0,G,PBFS74,331153676,0,5710,SAS,CABINET ROLET BONTEMPS,920240000Q0179
3415,A202101032527,2021-05-28,2527,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,PROGESCO,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2021-05-12,Jugement prononçant la liquidation judiciaire ...,682024542,92,1,004,ASNIERES-SUR-SEINE,000,AQ,0163,17.0,NaN,304.0,6660,RUE,DE NORMANDIE,210.0,NaN,S,210.0,S,PBBPVS,682024542,0,5710,SAS,PROGESCO SA,92004000AQ0163
3638,A202101232874,2021-06-25,2874,94,11.0,TRIBUNAL DE COMMERCE DE CRÉTEIL,ADB PATRIMOINE,Saint-Maur-des-Fossés,Jugemen

#### With locaux

In [96]:
loc_bodacc = df.merge(df_loc,
                      left_on='SIREN',
                      right_on = 'N° SIREN (Propriétaire(s) du local)',
                      how='left')

In [97]:
loc_bodacc = loc_bodacc[~loc_bodacc['N° SIREN (Propriétaire(s) du local)'].isna()]
loc_bodacc = loc_bodacc.drop_duplicates(['SIREN', 'jugement_date', 'Département (Champ géographique)', 
                                         'Code Commune (Champ géographique)', 'Section (Références cadastrales)',
                                         'Préfixe (Références cadastrales)', 'N° plan (Références cadastrales)'])

In [98]:
loc_bodacc

,id,dateparution,numeroannonce,numerodepartement,region_code,tribunal,commercant,ville,jugement_nature,jugement_famille,jugement_date,jugement_complement,SIREN,Département (Champ géographique),Code Direction (Champ géographique),Code Commune (Champ géographique),Nom Commune (Champ géographique),Préfixe (Références cadastrales),Section (Références cadastrales),N° plan (Références cadastrales),Bâtiment (Identification du local),Entrée (Identification du local),Niveau (Identification du local),Porte (Identification du local),N° voirie (Adresse du local),Indice de répétition (Adresse du local),Code voie MAJIC (Adresse du local),Code voie rivoli (Adresse du local),Nature voie (Adresse du local),Nom voie (Adresse du local),Code droit (Propriétaire(s) du local),N° MAJIC (Propriétaire(s) du local),N° SIREN (Propriétaire(s) du local),Groupe personne (Propriétaire(s) du local),Forme juridique (Propriétaire(s) du local),Forme juridique abrégée (Propriétaire(s) du local),Dénomination (Propriétaire(s) du local),parcelle_id
2,A200801921873,2008-10-22,1873,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,DATASCOR,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,29 septembre 2008,"Jugement prononçant la liquidation judiciaire,...",323505743,75,5,119,PARIS 19,000,AE,0001,A,03,0.0,1001.0,122.0,NaN,120.0,0517,RUE,D AUBERVILLIERS,P,PBDDGQ,323505743,0,5499,SARL,DATASCOR,75119000AE0001
9,A201002152048,2010-05-11,2048,93,11.0,TRIBUNAL DE COMMERCE DE BOBIGNY,DOCKS DE LA MALTOURNEE ETS F GUEGUEN ET CIE,Neuilly-sur-Marne,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,19 octobre 2010,"Jugement prononçant la liquidation judiciaire,...",558201760,93,0,066,SAINT DENIS,000,BY,0151,A,01,0.0,1001.0,29.0,B,81.0,0023,RUE,AMBROISE CROIZAT,P,PBBKFM,558201760,0,5599,SA,DOCKS LA MALTOURNEE ETS F GUEGUEN ET CIE,93066000BY0151
20,A201200313583,2012-02-14,3583,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,AGENCE ACTIVE,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,18 janvier 2012,"Jugement prononçant la liquidation judiciaire,...",328253158,59,2,459,PETITE FORET,000,AB,0248,A,01,0.0,1001.0,20.0,P,135.0,0886,VC,RES SIMONE DE BEAUVOIR,P,PBFP5M,328253158,0,5599,SA,AGENCE ACTIVE,59459000AB0248
22,A201201262179,2012-03-07,2179,75,11.0,TRIBUNAL DE COMMERCE DE PARIS,BAO,Paris,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,6 juin 2012,"Jugement prononçant la liquidation judiciaire,...",508608593,26,0,057,BOURG-DE-PEAGE,000,AH,0821,01,01,0.0,3.0,6.0,NaN,192.0,0745,ALL,DE PROVENCE,P,PBCXH3,508608593,0,5499,SARL,BAO,26057000AH0821
65,A201401342719,2014-07-16,2719,95,11.0,TRIBUNAL DE COMMERCE DE PONTOISE,SMS ENTREPOT,Persan,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,30 juin 2014,Jugement prononçant la liquidation judiciaire ...,532252228,95,0,487,PERSAN,000,AH,0017,I,01,0.0,1001.0,39.0,NaN,82.0,0170,RUE,DU DOCTEUR TOUATI,P,PBD6JQ,532252228,0,5499,SARL,SMS ENTREPOT,95487000AH0017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13672,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,000,AB,0010,A,01,0.0,1001.0,25.0,NaN,200.0,0160,GR,GRANDE RUE,P,PBCWT4,820222214,0,5710,SAS,SH LESPERANCE,89364000AB0010
13673,A202301002246,2023-05-25,2246,78,11.0,Greffe du Tribunal de Commerce de Versailles,SH L'ESPERANCE,Saint-Père,Jugement d'ouverture de liquidation judiciaire,Jugement d'ouverture de liquidation,2023-05-16,"Jugement prononçant la liquidation judiciaire,...",820222214,89,0,364,SAINT PERE,000,AB,0502,A,01,0.0,1001.0,22.0,NaN,200.0,0160,GR,GRANDE RUE,P,PBCWT4,820222214,0,5710

In [99]:
print('Number of locaux concerned :', len(loc_bodacc))

Number of locaux concerned : 404


In [100]:
loc_bodacc['Préfixe (Références cadastrales)'] = loc_bodacc['Section (Références cadastrales)'].apply(lambda x: '000' if len(x) == 2 else '0000')
df_loc['Préfixe (Références cadastrales)'] = df_loc['Section (Références cadastrales)'].apply(lambda x: '000' if len(str(x)) == 2 else '0000')

loc_bodacc['parcelle_id'] = loc_bodacc['Département (Champ géographique)'] + loc_bodacc['Code Commune (Champ géographique)']+ loc_bodacc['Préfixe (Références cadastrales)'] + loc_bodacc['Section (Références cadastrales)']  + loc_bodacc['N° plan (Références cadastrales)']
df_loc['parcelle_id'] = df_loc['Département (Champ géographique)'] + df_loc['Code Commune (Champ géographique)'] + df_loc['Préfixe (Références cadastrales)']  + df_loc['Section (Références cadastrales)'] + df_loc['N° plan (Références cadastrales)']

In [101]:
df_loc_perimetre = df_loc[df_loc['parcelle_id'].isin(loc_bodacc['parcelle_id'].unique())]
count_per_loc = df_loc_perimetre[['parcelle_id','N° SIREN (Propriétaire(s) du local)']].groupby('parcelle_id').count()
print('Proportion of locaux  where there is unique SIREN in it: ',
      round(100*len(count_per_loc[count_per_loc['N° SIREN (Propriétaire(s) du local)']!=1])/len(count_per_loc),2), '%' )

Proportion of locaux  where there is unique SIREN in it:  81.45 %


#### With SIRET

In [102]:
siret_bodacc = df.merge(SIRET, left_on='SIREN',
                               right_on = 'siren',
                               how='left')

In [103]:
len(siret_bodacc)

25969

In [104]:
siret_bodacc_loc = siret_bodacc.merge(loc_bodacc, how='left',
                                      left_on=['siren', 'libelleVoieEtablissement', 'libelleCommuneEtablissement'],
                                      right_on=['N° SIREN (Propriétaire(s) du local)',
                                                'Nom voie (Adresse du local)', 'Nom Commune (Champ géographique)'])

In [105]:
siret_bodacc_loc = siret_bodacc_loc[['siren', 'siret', 'libelleVoieEtablissement', 'libelleCommuneEtablissement',
                  'Code Commune (Champ géographique)', 'Nom Commune (Champ géographique)',
                  'Bâtiment (Identification du local)',
                  'Entrée (Identification du local)', 'Niveau (Identification du local)',
                  'Porte (Identification du local)', 'N° voirie (Adresse du local)',
                  'Indice de répétition (Adresse du local)',
                  'Code voie MAJIC (Adresse du local)', 'Code voie rivoli (Adresse du local)', 'Nature voie (Adresse du local)',
                  'Nom voie (Adresse du local)', 'Code droit (Propriétaire(s) du local)',
                  'N° MAJIC (Propriétaire(s) du local)',
                  'N° SIREN (Propriétaire(s) du local)',
                  'Groupe personne (Propriétaire(s) du local)',
                  'Forme juridique (Propriétaire(s) du local)',
                  'Forme juridique abrégée (Propriétaire(s) du local)',
                  'Dénomination (Propriétaire(s) du local)', 'parcelle_id']]
siret_bodacc_loc = siret_bodacc_loc[~siret_bodacc_loc['parcelle_id'].isna()]

In [106]:
len(siret_bodacc_loc)

30